In [2]:
import requests
import configparser
import snowflake.connector
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

In [3]:
# Load configuration
config = configparser.ConfigParser()
config.read("config.ini")

SNOWFLAKE_CONFIG = {
    "user": config["snowflake"]["user"],
    "password": config["snowflake"]["password"],
    "account": config["snowflake"]["account"],
    "warehouse": config["snowflake"]["warehouse"],
    "database": config["snowflake"]["database"],
    "schema": config["snowflake"]["schema"],
}

# Hugging Face API configuration for Mistral v2
API_URL = "https://api-inference.huggingface.co/models/mistralai/Mistral-7B-Instruct-v0.2"
API_KEY = config["huggingface"]["api_key"]

# Load the model with authentication (if required)
embedding_model = SentenceTransformer('all-MiniLM-L6-v2', use_auth_token=API_KEY)

HEADERS = {"Authorization": f"Bearer {API_KEY}", "Content-Type": "application/json"}

c:\Users\Ananya.Mehta\AppData\Local\Programs\Python\Python311\Lib\site-packages\sentence_transformers\SentenceTransformer.py:196: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v4 of SentenceTransformers.
  warnings.warn(


In [4]:
# Step 1: Create AI Solutions Table in Snowflake
def create_ai_solutions_table():
    conn = snowflake.connector.connect(**SNOWFLAKE_CONFIG)
    cursor = conn.cursor()
    
    query = """
    CREATE TABLE IF NOT EXISTS AI_SOLUTIONS (
        number STRING PRIMARY KEY,
        incident_subject STRING,
        priority STRING,
        business_service STRING,
        opened TIMESTAMP,
        short_description STRING,
        ai_generated_Solution STRING,
        root_cause_analysis STRING
    );
    """
    cursor.execute(query)
    conn.commit()
    cursor.close()
    conn.close()

In [5]:
# Step 2: Fetch Latest Email Alert from Snowflake
def fetch_latest_email_alert():
    conn = snowflake.connector.connect(**SNOWFLAKE_CONFIG)
    cursor = conn.cursor()
    
    query = """
        SELECT subject, INCIDENT_ID, impact_level, affected_service, timestamp, description
        FROM cleaned_alerts
        ORDER BY timestamp DESC
        LIMIT 1;
    """
    cursor.execute(query)
    latest_alert = cursor.fetchone()
    column_names = [desc[0].lower() for desc in cursor.description]
    cursor.close()
    conn.close()
    
    return dict(zip(column_names, latest_alert)) if latest_alert else None

In [13]:
def generate_root_cause_analysis(alert_data, similar_solution=None):
    if not alert_data:
        return "No new alerts found."
    
    formatted_alert = f"""
    Incident ID: {alert_data['incident_id']}
    Subject: {alert_data['subject']}
    Impact Level: {alert_data['impact_level']}
    Affected Service: {alert_data['affected_service']}
    Timestamp: {alert_data['timestamp']}
    Description: {alert_data['description']}
    """
    
    if similar_solution:
        prompt = f"""
        Analyze this alert and its similar solution to provide a detailed root cause analysis:\n\n{formatted_alert}\n\nSimilar Solution:\n{similar_solution}
        """
    else:
        prompt = f"""
        Analyze this alert and provide a detailed root cause analysis:\n\n{formatted_alert}
        """
    
    data = {"inputs": prompt}
    
    try:
        response = requests.post(API_URL, headers=HEADERS, json=data)
        response.raise_for_status()
        result = response.json()
        
        if isinstance(result, list) and result:
            rca_details = result[0].get("generated_text", "")
            if rca_details:  # Only print if analysis exists
                # Find the start of the root cause analysis section
                start_index = rca_details.find("Root Cause Analysis:")
                print(f"Root Cause Analysis for Incident {alert_data['incident_id']}:")
                if start_index != -1:
                    # Extract the root cause analysis section
                    rca_section = rca_details[start_index + len("Root Cause Analysis:"):].strip()
                    
                    # Remove leading/trailing whitespace and empty lines
                    rca_lines = [line.strip() for line in rca_section.splitlines() if line.strip()]
                    
                    # Format RCA details in bullets
                    formatted_rca = "\n".join(f"* {line}" for line in rca_lines)
                    print(formatted_rca)
                else:
                    print("No root cause analysis found.")
            return rca_details
        
        return ""
    
    except Exception as e:
        return ""


In [7]:
# Step 4: Store AI Solution in Snowflake
def store_rca_results(alert_data, root_cause_analysis):
    conn = snowflake.connector.connect(**SNOWFLAKE_CONFIG)
    cursor = conn.cursor()
    
    # Ensure this matches your RCA_RESULTS table schema
    query = """
        INSERT INTO RCA_RESULTS (
            NUMBER, CONFIGURATION_ITEM, INCIDENT_SUBJECT, BUSINESS_SERVICE, OPENED,
            ALERT_TIME, STATE, CLOSED_BY, CLOSED, PRIORITY,
            TICKET_OPENED_BY, INCIDENT_DURATION, ACTION_TO_RESOLVE_INCIDENT,
            RESOLUTION_TYPE, RESOLVED_BY, SHORT_DESCRIPTION,
            NEXT_STEPS, RCA
        )
        VALUES (
            %s, %s, %s, %s, %s,
            %s, %s, %s, %s, %s,
            %s, %s, %s, %s, %s,
            %s, %s, %s
        )
    """
    
    # Map alert_data fields to RCA_RESULTS columns
    values = (
        alert_data.get('incident_id', None),  # NUMBER
        alert_data.get('configuration_item', None),  # CONFIGURATION_ITEM
        alert_data.get('subject', None),  # INCIDENT_SUBJECT
        alert_data.get('business_service', None),  # BUSINESS_SERVICE
        alert_data.get('opened', None),  # OPENED
        alert_data.get('alert_time', None),  # ALERT_TIME
        alert_data.get('state', None),  # STATE
        alert_data.get('closed_by', None),  # CLOSED_BY
        alert_data.get('closed', None),  # CLOSED
        alert_data.get('priority', None),  # PRIORITY
        alert_data.get('ticket_opened_by', None),  # TICKET_OPENED_BY
        alert_data.get('incident_duration', None),  # INCIDENT_DURATION
        alert_data.get('action_to_resolve_incident', None),  # ACTION_TO_RESOLVE_INCIDENT
        alert_data.get('resolution_type', None),  # RESOLUTION_TYPE
        alert_data.get('resolved_by', None),  # RESOLVED_BY
        alert_data.get('short_description', None),  # SHORT_DESCRIPTION
        alert_data.get('next_steps', None),  # NEXT_STEPS
        root_cause_analysis  # RCA (Root Cause Analysis)
    )
    
    cursor.execute(query, values)
    conn.commit()
    cursor.close()


In [8]:
# Step 5: Load Existing Embeddings from Snowflake
def load_existing_embeddings():
    conn = snowflake.connector.connect(**SNOWFLAKE_CONFIG)
    cursor = conn.cursor()
    query = """
        SELECT number, short_description, ai_generated_Solution
        FROM AI_SOLUTIONS;
    """
    cursor.execute(query)
    rows = cursor.fetchall()
    cursor.close()

    incident_ids = [row[0] for row in rows]
    descriptions = [row[1] for row in rows]
    solutions = [row[2] for row in rows]

    return incident_ids, descriptions, solutions

In [9]:
# Step 6: Build FAISS Index and Perform Similarity Check
def build_faiss_index(descriptions):
   embeddings = embedding_model.encode(descriptions, convert_to_numpy=True)
   dim = embeddings.shape[1]
   index = faiss.IndexFlatL2(dim)
   index.add(embeddings)
   return index

def check_for_similar_alert(new_description, index, descriptions, incident_ids, solutions, threshold=0.85):
   new_embedding = embedding_model.encode([new_description], convert_to_numpy=True)
   D, I = index.search(new_embedding, k=1)
   similarity = 1 / (1 + D[0][0])  # Convert L2 to similarity
   
   if similarity >= threshold:
       match_idx = I[0][0]
       return solutions[match_idx]
   
   return None

In [14]:
# Main Workflow Execution
create_ai_solutions_table()

latest_alert = fetch_latest_email_alert()

if latest_alert:
   incident_ids, descriptions, solutions = load_existing_embeddings()

   if descriptions:
       index = build_faiss_index(descriptions)
       matched_solution = check_for_similar_alert(
           latest_alert["description"], index, descriptions, incident_ids, solutions
       )
   else:
       matched_solution = None

   if matched_solution:
       root_cause_analysis = generate_root_cause_analysis(latest_alert, matched_solution)
       store_rca_results(latest_alert, root_cause_analysis)
   else:
       generated_solution = generate_root_cause_analysis(latest_alert)  # Generate RCA without similar solution
       store_rca_results(latest_alert, generated_solution)  # Store RCA in new table


Root Cause Analysis for Incident INC-20250407-002:
* Both incidents describe unauthorized access attempts, but they differ in their contexts and methods.
* The first incident is about an attempt to access the primary database cluster directly through its network interface. In this case, an unrecognized IP address attempted multiple failed login attempts, infringing on the basic security principle that only authorized entities should be allowed to access the database.
* The second incident is about an API request attempt, which failed due to an Access Denied error. The request was trying to create a new user with the "admin" role, which likely required permissions beyond those granted to the requester.
* Although these incidents do not appear to be directly related, they highlight the importance of maintenance, updates, and strict access control measures.
* In the first incident, attacks against the database network may indicate vulnerabilities that attackers can exploit, such as outdat